In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical, plot_model
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


train = pd.read_csv('../input/Kannada-MNIST/train.csv')

Y_train = np.array(train['label'])
X_train = train.drop(['label'],1)
X_train = np.array(X_train).reshape(X_train.shape[0],28,28)
X_train = X_train.reshape(-1, 28, 28, 1)
Y_train = to_categorical(Y_train, 10)
X_train = np.array(X_train/255)

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

imagegen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,
        zca_whitening=False, 
        rotation_range=9, 
        zoom_range = 0.25,
        width_shift_range=0.25,
        height_shift_range=0.25, 
        horizontal_flip=False, 
        vertical_flip=False)

imagegen.fit(x_train)

In [ ]:
#CNN Solution

#Fitting Parameters
epochs = 10
batch_size = 32
lr = 0.001
dropout = 0.40

cnn = Sequential()
cnn.add(Conv2D(filters=64, kernel_size=3, padding = 'same', activation='relu', input_shape=(28, 28, 1)))
cnn.add(Conv2D(filters=64, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(Conv2D(filters=128, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(Conv2D(filters=128, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(Conv2D(filters=256, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(MaxPooling2D(2))
cnn.add(Conv2D(filters=512, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(MaxPooling2D(2))
#cnn.add(Conv2D(filters=1024, kernel_size=3, padding = 'same', activation='relu'))
cnn.add(Flatten())
cnn.add(Dropout(dropout))
cnn.add(Dense(200, activation = 'relu'))
cnn.add(Dropout(dropout))
cnn.add(Dense(10, activation='softmax'))
#plot_model(cnn, show_shapes=True, to_file="modelcnn.png")

cnn.compile(loss='categorical_crossentropy', optimizer=adam(lr=lr), metrics=['accuracy'])
checkpoint=ModelCheckpoint('bestweights.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode="max")
'''
hist_cnn = cnn.fit(x_train, y_train, 
                   batch_size = batch_size, 
                   epochs = epochs, 
                  verbose = 1,
                  validation_split = 0.2,
                  callbacks = [checkpoint])
'''
hist_cnn = cnn.fit_generator(imagegen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val,y_val),
                              verbose = 1, steps_per_epoch=x_train.shape[0]/10,
                             callbacks = [checkpoint])

cnn.load_weights("bestweights.hdf5")

In [ ]:
#graph accuracy and loss
fig = plt.figure()
fig.patch.set_facecolor('white')
fig.set_size_inches(10,5)

plt.subplot(1, 2, 1)
plt.plot(hist_cnn.history['acc'])
plt.plot(hist_cnn.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(hist_cnn.history['loss'])
plt.plot(hist_cnn.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')

plt.show()

In [ ]:
test = pd.read_csv('../input/Kannada-MNIST/test.csv')

test = test.drop(['id'],1)
x_test = np.array(test/255)
x_test = x_test.reshape(x_test.shape[0],28,28)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

results=cnn.predict(x_test)
results=np.argmax(results, axis=1)
cnn3 = pd.DataFrame({'label':results}).reset_index().rename(columns = {'index':'id'})
cnn3.to_csv('cnn3_img_aug.csv',index = False)

